In [19]:
import os
from dotenv import load_dotenv
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq

In [12]:
load_dotenv()

True

In [2]:
bge_embedding = HuggingFaceBgeEmbeddings(
    model_name='BAAI/bge-large-en-v1.5',
    model_kwargs={"device": "cpu"},
)

C:\Users\MSUSERSL123\AppData\Local\Temp\ipykernel_12180\4210569487.py:1: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  bge_embedding = HuggingFaceBgeEmbeddings(


In [3]:
vector_store = Chroma(
    persist_directory="./bge_chroma_store",
    embedding_function=bge_embedding
)

C:\Users\MSUSERSL123\AppData\Local\Temp\ipykernel_12180\3711123431.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vector_store = Chroma(


In [25]:
def format_docs(docs):
    return "/n/n".join([doc.page_content for doc in docs])

In [6]:
retriever = vector_store.as_retriever()

In [15]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key=os.getenv('GROQ_API_KEY'),
    temperature=0,
    model_kwargs={
        "top_p": 1    
    }
)

In [17]:
prompt = PromptTemplate(input_varibale=["firm_style", "context"], template="""
You are an expert proposal writer. Your task is to generate the “Executive Summary” section for a Digital Marketing proposal.

Use the following two inputs:
1. CONTEXT: Executive summary fetched from various real proposals example related to digital marketing.
2. FIRM_STYLE: Real proposal examples from the consulting firm. These may be unrelated to Digital Marketing, but must be used to infer tone and writing patterns.

--- STRICT INSTRUCTIONS ---

• First, analyze FIRM_STYLE to understand:
  - Tone (formal, concise, business-oriented, narrative, etc.)
  - Writing structure (bullets vs. paragraphs, storytelling patterns)
  - Vocabulary and phrasing patterns

• Then generate the Executive Summary for Digital Marketing by:
  - Learning section length (short/long, number of paragraphs) from given CONTEXT
  - Learn how different firms are actually writing executive summary
  - Tone + structure learned from FIRM_STYLE

• Do NOT copy text from FIRM_STYLE. Only learn style and format.

• Executive Summary must:
  - Clearly articulate client’s situation, challenges, and desired outcomes
  - Present a high-level view of the digital marketing solution
  - Reflect the same level of polish as a real firm proposal
  - Match the expected length based on CONTEXT and FIRM_STYLE

• If CONTEXT lacks some information, keep the narrative general without inventing specifics.

--- OUTPUT FORMAT ---
Write only the “Executive Summary” section.
Do not include headings like CONTEXT ANALYSIS or STYLE ANALYSIS in your output.

CONTEXT:
{context}

FIRM_STYLE:
{firm_style}
""")

In [28]:
chain = (RunnableParallel({
    'firm_style': lambda x: x['firm_style'],
    'context': lambda x: format_docs(retriever.invoke(x['about']))
})
| prompt
| llm
| StrOutputParser)

In [21]:
about = "We are a performance-focused digital marketing firm specializing in end-to-end growth strategies across SEO, paid media, content, and analytics. Our team blends creative storytelling with data intelligence to drive measurable ROI and long-term brand visibility. We work closely with clients to understand their audience, optimize digital activation, and accelerate engagement across channels. Our solutions are tailored, scalable, and aligned with each client’s evolving market objectives."
firm_style = "Our firm partners with clients to navigate complex business challenges through data-driven insights and structured problem-solving. We emphasize clarity, measurable outcomes, and a collaborative approach that aligns strategic priorities with operational execution. Our proposals maintain a concise narrative supported by evidence-based recommendations. Each engagement is designed to deliver sustainable value with a clear roadmap for implementation."

In [29]:
ans = chain.invoke({'firm_style': firm_style, 'about': about})

TypeError: BaseModel.__init__() takes 1 positional argument but 2 were given